### Testa att bygga med scentence transformer istället för gemini

In [3]:
import json

with open("data/yrkesbarometer (1).json", encoding="utf-8") as file:
    data = json.loads(file.read())

data

[{'yrkesomrade': 'Administration, ekonomi, juridik',
  'yb_yrke': 'Administrativa assistenter',
  'yb_concept_id': 'PDNm_NZ2_WRh',
  'lan': '00',
  'antal_ssyk': 1,
  'del_av_ssyk': 0,
  'ssyk': '4119',
  'ssyk_text': 'Övriga kontorsassistenter och sekreterare',
  'yb_beskrivning': None,
  'jobbmojligheter': 'små',
  'rekryteringssituation': 'överskott',
  'paradox': [],
  'prognos': 'vara oförändrad',
  'text_jobbmojligheter': 'Nationellt bedöms möjligheterna till arbete som administrativ assistent vara små. På fem års sikt förväntas efterfrågan vara oförändrad.',
  'text_rekryteringssituation': 'Nationellt bedöms rekryteringssituationen för administrativa assistenter kännetecknas av överskott. På fem års sikt förväntas efterfrågan på administrativa assistenter vara oförändrad.',
  'hogsta_bedomningsniva': 'nationellt',
  'delvis_helt': 'helt',
  'omgang': '2025-2',
  'taxonomi_version': 27},
 {'yrkesomrade': 'Administration, ekonomi, juridik',
  'yb_yrke': 'Administrativa assistenter

In [4]:
import pandas as pd

df = pd.DataFrame(data)
df.shape

(4818, 19)

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

vec = model.encode(["Hej världen!"])
print(type(vec), vec.shape)


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 509.57it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


<class 'numpy.ndarray'> (1, 384)


In [8]:
#Bygger ett dokument av en enskild rad i dataframen

row = df.iloc[0]          

def build_document_simple(row) -> str:
    return f"""
Yrke: {row['yb_yrke']}
Yrkesområde: {row['yrkesomrade']}
Län: {row['lan']}

{row['text_jobbmojligheter']}
{row['text_rekryteringssituation']}

""".strip()

doc = build_document_simple(row)
print(doc)

Yrke: Administrativa assistenter
Yrkesområde: Administration, ekonomi, juridik
Län: 00

Nationellt bedöms möjligheterna till arbete som administrativ assistent vara små. På fem års sikt förväntas efterfrågan vara oförändrad.
Nationellt bedöms rekryteringssituationen för administrativa assistenter kännetecknas av överskott. På fem års sikt förväntas efterfrågan på administrativa assistenter vara oförändrad.


In [ ]:
#Embedda texten av den enskilda raden ovan (doc)

vec = model.encode([doc])
print(vec.shape)

(1, 384)


In [13]:
#Bygger dokumnet för tre de tre första raderna i dataframen

docs = []
for _, row in df.head(3).iterrows():
    docs.append(build_document_simple(row))

len(docs), docs[0][:200]

(3,
 'Yrke: Administrativa assistenter\nYrkesområde: Administration, ekonomi, juridik\nLän: 00\n\nNationellt bedöms möjligheterna till arbete som administrativ assistent vara små. På fem års sikt förväntas efte')

In [14]:
#Embedda texten av samtliga tre dokument (docs)

vectors = model.encode(docs)
print(vectors.shape)

(3, 384)


In [15]:
#Lägga in i en vektordatabas - mindre urval av dokument

df_vec = df.head(50).copy()

df_vec["document"] = df_vec.apply(build_document_simple, axis=1)

vectors = model.encode(df_vec["document"].tolist())
df_vec["vector"] = [v.tolist() for v in vectors]

df_vec[["yb_yrke", "lan", "vector"]].head(1)



,yb_yrke,lan,vector
0,Administrativa assistenter,00,"[0.01730147749185562, 0.03300710394978523, 0.0..."


In [16]:
import lancedb


db = lancedb.connect("rag_playground/db/yrkesbarometer_vectors")

table = db.create_table(
    "yrken",
    df_vec[["yb_yrke", "lan", "document", "vector"]],
    mode="overwrite"
)

table.count_rows()


50

In [19]:
query = "yrken inom IT med goda framtidsutsikter"
query_vec = model.encode([query])[0].tolist()

results = (
    table.search(query_vec)
    .limit(5)
    .select(["yb_yrke", "lan", "_distance"])
    .to_pandas()
)

results


,yb_yrke,lan,_distance
0,Arbetsförmedlare,00,0.970733
1,Arbetsförmedlare,14,0.995424
2,Arbetsförmedlare,12,1.006532
3,Arbetsförmedlare,21,1.174815
4,Arbetsförmedlare,24,1.186484


In [ ]:
#


df_it_small = df[(df["yrkesomrade"] == "Data/IT") & (df["lan"] == "00")].copy()
df_it_small = df_it_small.head(200)  # lagom för test
df_it_small["document"] = df_it_small.apply(build_document_simple, axis=1)

vecs = model.encode(df_it_small["document"].tolist(), show_progress_bar=True)
df_it_small["vector"] = [v.tolist() for v in vecs]

table_it = db.create_table(
    "yrken_it",
    df_it_small[["yb_yrke", "lan", "document", "vector"]],
    mode="overwrite"
)

print("rows:", table_it.count_rows())


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.80it/s]

rows: 10


In [21]:
query = "yrken inom IT med goda framtidsutsikter"
q_vec = model.encode([query])[0].tolist()

results = (
    table_it.search(q_vec)
    .limit(5)
    .select(["yb_yrke", "lan", "_distance"])
    .to_pandas()
)
results


,yb_yrke,lan,_distance
0,Nätverks- och systemtekniker,00,1.128842
1,Supporttekniker inom IT,00,1.131399
2,Drifttekniker inom IT,00,1.168229
3,Systemförvaltare,00,1.189614
4,IT-säkerhetsspecialister,00,1.195391
